### 05_대화 메모리 관리

In [2]:
import os
from dotenv import load_dotenv

In [3]:
load_dotenv("/home/ubuntu/project/.env")

True

In [4]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage

In [5]:
# 1) 메모리 객체 생성
history = InMemoryChatMessageHistory()

In [6]:
# 2) 사용자 메시지 추가
history.add_message(HumanMessage(content="안녕?"))

In [7]:
# 3) AI 응답 추가
history.add_message(AIMessage(content="안녕하세요! 만나서 반가워요."))

In [8]:
# 4) 기록 확인
print("=== 대화 기록 ===")
for msg in history.messages:
    role = "🧑 사용자" if msg.type == "human" else "🤖 AI"
    print(f"{role}: {msg.content}")

# 프로그램이 종료되면 기록이 사라지는 휘발성(volatile) 구조

=== 대화 기록 ===
🧑 사용자: 안녕?
🤖 AI: 안녕하세요! 만나서 반가워요.


In [9]:
pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.


In [10]:
# 대화 히스토리를 파일로 (JSON)으로 저장하는 법
from langchain_community.chat_message_histories import FileChatMessageHistory

In [11]:
from langchain_core.messages import HumanMessage, AIMessage

In [12]:
#1) FileChatMessageHistory 생성
history = FileChatMessageHistory("chat_history_json")

In [13]:
#2) 사용자 메시지 추가
history.add_message(HumanMessage(content="안녕?"))
history.add_message(AIMessage(content="안녕하세요! 만나서 반가워요."))

# 3) 현재 메시지 확인
print("=== 대화 이력 ===")
for msg in history.messages:
    role = "🧑 사용자" if msg.type == "human" else "🤖 AI"
    print(f"{role}: {msg.content}")

# 4) 프로그램 종료 후 다시 실행해도 chat_history.json 파일에 저장된 내용이 유지됨


=== 대화 이력 ===
🧑 사용자: 안녕?
🧑 사용자: 안녕?
🤖 AI: 안녕하세요! 만나서 반가워요.
🧑 사용자: 안녕?
🤖 AI: 안녕하세요! 만나서 반가워요.
🧑 사용자: 안녕?
🤖 AI: 안녕하세요! 만나서 반가워요.


In [14]:
dir

<function dir>

In [15]:
# 러너블 객체와 함께 연동하는 법
# 요즘 스타일: Conversation Chain -> RunnableWithMessageHistory 로 변경
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory 

In [16]:
# [1] LLM 모델 정의
llm = ChatOpenAI(model = "gpt-4o-mini", temperature = 0.3)

In [17]:
# [2] session별 대화 기록 저장소
store = {}  #왜 딕셔너리인지?

In [18]:
def get_session_history(session_id: str):
    """ 세션 id 별로 대화 기록을 저장하고 불러오는 함수"""
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]
    

In [19]:
# [3] RunnableWithMessageHistory 구성
conversation = RunnableWithMessageHistory(
    llm,
    get_session_history
)

In [20]:
# [4] session id 설정
config = {"configurable":{"session_id":"chat-001"}}

In [21]:
response1 = conversation.invoke([HumanMessage(content="안녕? 나는 홍길동이야!")], config=config)
print("🤖:", response1.content)

response2 = conversation.invoke([HumanMessage(content="내 이름이 뭐지?")], config=config)
print("🤖:", response2.content)

# 이 코드를 실행하면 history_user-001.json 파일이 자동으로 생성되고,
# 모든 대화 내용이 파일에 순차적으로 누적 저장됩니다.

🤖: 안녕하세요, 홍길동님! 만나서 반갑습니다. 어떻게 도와드릴까요?
🤖: 당신의 이름은 홍길동입니다! 다른 질문이나 도움이 필요하신 것이 있나요?


In [ ]:
# [5] RedisChatMessageHistory
# 메모리에 휘발되는 InMemoryChatMessageHistory 대신 Redis DB 서버에 대화 내용을 저장해서 
# 프로그램을 꺼도, 혹은 여러 사용자가 동시에 접속해도 이전 대화가 유지되는 구조

In [22]:
!pip install redis

In [23]:
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage

In [24]:
# 1) Redis 서버 연결 설정
# host, port, session_id는 자유롭게 변경 가능
history = RedisChatMessageHistory(
    url="redis://localhost:6379/0",  # Redis 서버 주소
    session_id="user-001"            # 세션 구분용 ID
)

In [26]:
# 2) 메시지 추가
history.add_message(HumanMessage(content="안녕?"))
history.add_message(AIMessage(content="안녕하세요! 반가워요"))

In [27]:
# 3) 현재 대화 이력 확인
print("=== Redis 저장된 대화 이력 ===")
for msg in history.messages:
    role = "사용자" if msg.type == "human" else "AI"
    print(f"{role}: {msg.content}")

# Redis 내부에 다음과 같이 저장됨
# KEY: chat_history:user-001
# VALUE: [
#   {"type": "human", "content": "안녕?"},
#   {"type": "AI", "content": "안녕하세요! 반가워요"}
# ]

=== Redis 저장된 대화 이력 ===
사용자: 안녕?
AI: 안녕하세요! 반가워요


In [28]:
# Redis에 저장된 키와 내용 확인하기 
import redis
import json

#  Redis 연결
r = redis.Redis(host='localhost', port=6379, db=0)

#  모든 키 확인
keys = r.keys('*')

if not keys:
    print("❌ 현재 Redis에 저장된 키가 없습니다.")
else:
    print("📦 Redis에 저장된 키와 내용:")
    print("=" * 50)

    for k in keys:
        key = k.decode()
        dtype = r.type(k).decode()

        print(f"🔑 {key}  |  타입: {dtype}")

        if dtype == "list":
            messages = r.lrange(k, 0, -1)
            for i, msg in enumerate(messages, 1):
                try:
                    data = json.loads(msg)
                    msg_type = data.get("type", "")
                    content = data.get("data", {}).get("content", "")
                    # 보기 좋은 출력 포맷
                    print(f"  💬 {i}. {{")
                    print(f'    "type": "{msg_type}",')
                    print(f'    "data": {{"content": "{content}"}}')
                    print("  }")
                except Exception:
                    print(f"  💬 {i}. {msg.decode(errors='ignore')}")
            print("-" * 60)


📦 Redis에 저장된 키와 내용:
🔑 message_store:user-001  |  타입: list
  💬 1. {
    "type": "ai",
    "data": {"content": "안녕하세요! 반가워요"}
  }
  💬 2. {
    "type": "human",
    "data": {"content": "안녕?"}
  }
------------------------------------------------------------


In [29]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# 1) LLM 모델 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.4)

# 2) Redis 기반 대화 저장소 함수
def get_session_history(session_id: str):
    return RedisChatMessageHistory(
        url="redis://localhost:6379/0",
        session_id=session_id
    )

# 3) RunnableWithMessageHistory 생성
conversation = RunnableWithMessageHistory(llm, get_session_history)

# 4) 세션 설정
config = {"configurable": {"session_id": "chat-redis-001"}}

# 5) 대화 실행
response1 = conversation.invoke([HumanMessage(content="안녕? 나는 홍길동이야!")], config=config)
print("🤖:", response1.content)

response2 = conversation.invoke([HumanMessage(content="내 이름이 뭐지?")], config=config)
print("🤖:", response2.content)

# 6) Redis에 저장된 대화 확인
history = get_session_history("chat-redis-001")
print("\n=== Redis 저장된 대화 이력 ===")
for msg in history.messages:
    role = "🧑 사용자" if msg.type == "human" else "🤖 AI"
    print(f"{role}: {msg.content}")


🤖: 안녕하세요, 홍길동님! 만나서 반갑습니다. 어떻게 도와드릴까요?
🤖: 당신의 이름은 홍길동입니다! 다른 질문이나 궁금한 점이 있으신가요?

=== Redis 저장된 대화 이력 ===
🧑 사용자: 안녕? 나는 홍길동이야!
🤖 AI: 안녕하세요, 홍길동님! 만나서 반갑습니다. 어떻게 도와드릴까요?
🧑 사용자: 내 이름이 뭐지?
🤖 AI: 당신의 이름은 홍길동입니다! 다른 질문이나 궁금한 점이 있으신가요?


In [ ]:
### redis 를 쓰고 나면 어딘가 백그라운드에서 돌아가고 있을 것
### 터미널에서 끄고 켜는 것 가능
# sudo systemctl stop redis-server
# sudo systemctl status redis-server

# 다시 켜는 방법
# sudo systemctl start redis-server